<a href="https://colab.research.google.com/github/dewansh2004/Data-Scientis-tools/blob/main/Binarization_on_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [33]:
# The accuracy score decreased after binarization because the Binarizer transformed the numerical 'family' feature into a binary one (0 or 1).
# This process led to a loss of information, as the model could no longer distinguish between different positive counts of family members
# (e.g., 1 family member vs. 5 family members). A Decision Tree model might have benefited from the richer, numerical representation
# of the 'family' feature to make more nuanced decisions. Simplifying it to a binary value reduced its ability to capture those
# patterns effectively, thus leading to a lower accuracy.
df = pd.read_csv('/content/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
df = pd.read_csv('/content/train.csv')[['Age','Fare','SibSp','Parch','Survived']]

In [13]:
df.dropna(inplace=True)

In [16]:
df['family'] = df['SibSp']+df['Parch']

In [17]:
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [19]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [20]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [21]:
x = df.drop(columns=['Survived'])
y = df['Survived']

In [23]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [24]:
x_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [25]:
#without binarization

clf = DecisionTreeClassifier()

clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

accuracy_score(y_test,y_pred)

0.6153846153846154

In [26]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

np.float64(0.6513302034428795)

In [27]:
#applying binarization

from sklearn.preprocessing import Binarizer

In [29]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

In [30]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [31]:
pd.DataFrame(x_train_trf,columns=['family','age','fare'])

,family,age,fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [32]:
clf = DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
y_pred2 = clf.predict(x_test_trf)

accuracy_score(y_test,y_pred2)

0.5944055944055944